In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Install necessary libraries

In [ ]:
!pip install -q "monai-weekly[gdown, nibabel, tqdm, itk]"
!pip install pydicom
!pip install torch # install if not available

# Load necessary libraries

In [ ]:
import os
import shutil
import tempfile
import matplotlib.pyplot as plt
import PIL
from PIL import Image
import torch
import numpy as np
from monai.config import print_config
from monai.networks.nets import DenseNet121
from monai.utils import set_determinism
import pandas as pd
import pydicom
import glob
import cv2 as cv
import os
import torch
from torch.utils.data import Dataset, DataLoader

from monai.transforms import (
    Activations,
    AddChannel,
    AsDiscrete,
    Compose,
    LoadImage,
    RandFlip,
    RandRotate,
    RandZoom,
    ScaleIntensity,
    ToTensor,
)

def dcmtopng(test_dicoms):
  outdir = '/content/drive/MyDrive/AI Girls/ai_wonder_girls_medical_imaging/Resources/testdicomToPNG/'

  for f in test_dicoms:
      ds = pydicom.read_file(f) # read dicom image
      img = ds.pixel_array # get image array
      dirName = f.split("/")[-3:][0] + "/" + f.split("/")[-3:][1] + "/"
      try:
        os.makedirs(outdir + dirName)
      except:
        pass
      cv.imwrite(outdir + dirName + f.split("/")[-1].replace('.dcm','.png'), img) # write png 
  return outdir

def preprocess_imgs(test_png_f):
  for f in test_png_f:
      im = cv.imread(f)
      im = cv.resize(im, (224, 224))
      final_img = cv.normalize(im,  im, 0, 255, cv.NORM_MINMAX)
      cv.imwrite(f, final_img[:, :, 1]) # save image in gray format 

def get_dcm_data(test_dicoms):
  all_data = []

  for i in test_dicoms: 
      dcm = pydicom.dcmread(i)
      fields = [dcm.PatientID, int(dcm.PatientAge[1:3]), dcm.PatientSex, dcm.Modality, dcm.StudyDescription, dcm.BodyPartExamined, 
              dcm.StudyDate, dcm.AcquisitionDate, dcm.InstitutionName]
      all_data.append(fields)
  mydata = pd.DataFrame(all_data, 
                        columns = ['PatientID','Age','Gender','Modality','Study Description', 'Body Part','Study Date','Acquisition Date', 'Institution Name'])
  mydata["Path to file"] = test_png_f

  mydata['Study Description'] = mydata['Study Description'].map(lambda x : x.replace('Handgelenk links', 'Left wrist')
                                                          .replace('Handgelenk rechts','Right wrist'))
  mydata['Study Date'] = pd.to_datetime(mydata['Study Date'])

  csv_path = dcm_png_dir + "test_data.csv"
  mydata.to_csv(csv_path)
  return csv_path

class HandFracDataset(Dataset):

    def __init__(self, image_files, transforms):
        self.image_files = image_files
        self.transforms = transforms

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, index):
        return self.transforms(self.image_files[index])

def infer(model, weight_path):
  model.load_state_dict(torch.load(weight_path, map_location=lambda storage, loc: storage))
  model.eval()
  y_pred = list()
  with torch.no_grad():
    for test_data in test_loader:
        test_images = test_data.to(device)
        pred = model(test_images).argmax(dim=1)
        for i in range(len(pred)):
            y_pred.append(pred[i].item())
  return y_pred

# Provide test dir 
## Structure is: 
## - test_set
## -- folder_of_dicom_file 
## --- dicom_file
test_dir = '/content/drive/MyDrive/AI Girls/ai_wonder_girls_medical_imaging/Resources/university-hospital-basel/testing_set/'
test_dicoms = glob.glob(test_dir + "*/*.dcm")

# Get the dir storing png files
outdir = dcmtopng(test_dicoms)
dcm_png_dir = outdir
test_png_f = glob.glob(dcm_png_dir + "*/*/*.png")

# Preprocess the png files
preprocess_imgs(test_png_f)

# Get dicom data
test_df = pd.read_csv(get_dcm_data(test_dicoms))

class_names = ["yes", "no"]
num_class = len(class_names)

image_file_list = test_df["Path to file"]
image_width, image_height = Image.open(image_file_list[0]).size

# Perform transforms
val_transforms = Compose([
    LoadImage(image_only=True),
    AddChannel(),
    ScaleIntensity(),
    ToTensor()
])

act = Activations(softmax=True)

# Read test data as batch
test_ds = HandFracDataset(image_file_list, val_transforms)
test_loader = DataLoader(test_ds, batch_size=32, num_workers=2)

# define network
device = torch.device('cpu')
model = DenseNet121(
  spatial_dims=2,
  in_channels=1,
  out_channels=num_class, 
  pretrained = True).to(device)
loss_function = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), 1e-4)#Adam 1e-5
epoch_num = 30
val_interval = 1

# Perform prediction
weight_path = '/content/drive/MyDrive/AI Girls/ai_wonder_girls_medical_imaging/Resources/modeling/model_outputs/best_metric_model_v0f.pth'
y_pred = infer(model, weight_path)

# Store prediction in test df
test_df["prediction"] = [class_names[pred] + " fracture" for pred in y_pred]
test_df.drop(columns="Unnamed: 0", inplace=True)
# Save df file as csv
test_df.to_csv(dcm_png_dir + "test_data_pred-rosana.csv")

Downloading: "https://download.pytorch.org/models/densenet121-a639ec97.pth" to /root/.cache/torch/hub/checkpoints/densenet121-a639ec97.pth


  0%|          | 0.00/30.8M [00:00<?, ?B/s]